In [3]:
%load_ext autoreload
%autoreload 2
# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar ç
from selenium.webdriver.common.by import By

from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import re

#Librerías para hacer la ELT
import sys
sys.path.append("../../")
from src import soporte_sql as s_sql
from src import metodos_limpieza_df as s_limpieza

#opciones vis

# pd.set_option("display.max_columns",None)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Vamos a crear las tablas de la bdd

In [4]:
conn = s_sql.crear_conexion(bdd_local=True)
cur = conn.cursor()

# Asegurar que no haya transacciones bloqueadas
conn.rollback()  

crear_tablas = """
    
    DROP TABLE IF EXISTS modelos CASCADE;
    CREATE TABLE modelos (
        marca TEXT,
        modelo TEXT,
        anio INTEGER,
        primary key (marca, modelo, anio),
        tipo_carroceria TEXT,
        link TEXT
    );


    DROP TABLE IF EXISTS caracteristicas;
    CREATE TABLE caracteristicas (
        marca TEXT,
        modelo TEXT,
        anio INTEGER,
        primary key (marca, modelo, anio),
        precio_base NUMERIC,
        precio_max NUMERIC,
        fecha_inicio DATE,
        fecha_fin DATE,
        potencia_minima NUMERIC,
        potencia_maxima NUMERIC,
        lon_0 NUMERIC,
        lon_1 NUMERIC,
        maletero_0 NUMERIC,
        maletero_1 NUMERIC,
        cambio_man INT,
        cambio_aut INT,
        tracc_del INT,
        tracc_total INT,
        tracc_tras INT,
        diesel INT,
        gasolina INT,
        electrificado INT,
        FOREIGN KEY (marca, modelo, anio) REFERENCES modelos (marca, modelo, anio)
    );

    DROP TABLE IF EXISTS reviews;
    CREATE TABLE reviews (
        marca TEXT,
        modelo TEXT,
        anio INTEGER,
        primary key (marca, modelo, anio),
        reportaje TEXT,
        FOREIGN KEY (marca, modelo, anio) REFERENCES modelos (marca, modelo, anio)
    );
"""


try:
    cur.execute(crear_tablas)
    conn.commit()
    print("Tablas creadas correctamente ✅")
except Exception as e:
    conn.rollback()  # Si hay error, cancelar transacción
    print("Error al crear el trigger tablas:", e)
finally:
    cur.close()
    conn.close()


la contraseña es errónea


AttributeError: 'NoneType' object has no attribute 'cursor'

In [11]:
conn = s_sql.crear_conexion()
cur = conn.cursor()

# Asegurar que no haya transacciones bloqueadas
conn.rollback()  

crear_vectorial = """
CREATE EXTENSION IF NOT EXISTS vector;

DROP TABLE IF EXISTS documentos CASCADE;
CREATE TABLE documentos (
    id SERIAL PRIMARY KEY,
    contenido TEXT,
    embedding vector(1536)  
);
"""

try:
    cur.execute(crear_vectorial)
    conn.commit()
    print("Tablas creadas correctamente ✅")
except Exception as e:
    conn.rollback()  # Si hay error, cancelar transacción
    print("Error al crear las tablas:", e)
finally:
    cur.close()
    conn.close()

Conexión creada con éxito
Error al crear las tablas: la extensión «vector» no está disponible
DETAIL:  No se pudo abrir el archivo de control de extensión «C:/Program Files/PostgreSQL/17/share/extension/vector.control»: No such file or directory.
HINT:  La extensión debe primero ser instalada en el sistema donde PostgreSQL está ejecutándose.



FUNCIONES

In [4]:
def procesar_link(enlace):
    if len(enlace.split("/")) == 7:
        dic_coche = {
            "marca": enlace.split("/")[2],
            "modelo": enlace.split("/")[3],
            "anio": enlace.split("/")[4],
            "tipo_carroceria": enlace.split("/")[5],
            "link": enlace
        }
    elif len(enlace.split("/")) == 8:
        dic_coche = {
            "marca": enlace.split("/")[2],
            "modelo": f"{enlace.split('/')[3]} {enlace.split('/')[6]}",
            "anio": enlace.split("/")[4],
            "tipo_carroceria": enlace.split("/")[5],
            "link": enlace
        }
    else:
        return None

    # Limpiar texto
    for key in ["marca", "modelo", "anio", "tipo_carroceria"]:
        dic_coche[key] = s_limpieza.limpiar_texto(dic_coche[key])

    return dic_coche

def insertar_en_bd(datos):
    conn = s_sql.crear_conexion()
    cur = conn.cursor()

    if not datos:
        return
    insert_query = """
        INSERT INTO modelos (marca, modelo, anio, tipo_carroceria, link)
        VALUES (%s, %s, %s, %s, %s)
        ON CONFLICT (marca, modelo, anio)
        DO UPDATE SET tipo_carroceria = EXCLUDED.tipo_carroceria, link = EXCLUDED.link;
    """
    cur.executemany(insert_query, [(d["marca"], d["modelo"], d["anio"], d["tipo_carroceria"], d["link"]) for d in datos])
    conn.commit()

    conn.close()
    cur.close()

def limpiar_valores_diccionario(diccionario):
    resultado = {}
    for clave, valor in diccionario.items():
        # Buscar números, decimales o separados por comas/puntos
        numeros = re.findall(r'[\d.,]+', valor)
        if numeros:
            # Tomamos el primer número encontrado como valor limpio
            try:
                num_limpio = numeros[0].replace(".", "").replace(",", ".")
                resultado[clave] = num_limpio
                
            except clave == "Tarifa de":
                pass

        else:
            # Si no hay números, dejamos el valor original
            resultado[clave] = valor
    return resultado


En primer lugar lo que haremos será buscar todas las URLs de los coches a escrapear. Contruimos un df con las mismas y lo subimos directamente a Dbeaver

In [ ]:
# Crear conexión a PostgreSQL

BASE_URL = "https://www.km77.com/buscador/informaciones?grouped=0&order=date-desc&markets[]=current&markets[]=discontinued"

response_num_pags = requests.get(BASE_URL)
if response_num_pags.status_code == 200:

    sopa_pags = BeautifulSoup(response_num_pags.content, "html.parser")
    pags_totales = sopa_pags.find("div", class_="d-inline font-weight-normal font-size-2xl").text.strip()
    total_resultados = int(re.findall(r"\d+", pags_totales)[0])
    tot_pags =  (total_resultados // 20) + 1  
    print(f"Hay {tot_pags} paginas")# Cálculo de páginas

else:
    print("Error al obtener el total de páginas")


# Función para obtener los links de una página 
if tot_pags:
    for numero_pagina in range(0,tot_pags+1):
        url_review = f"{BASE_URL}&page={numero_pagina}"

        try:
            res = requests.get(url_review) 
            if res.status_code == 200:
                sopa_review = BeautifulSoup(res.content, "html.parser")
                titulo = sopa_review.find("div", class_="d-inline font-weight-normal font-size-2xl")
                    
                if titulo.get_text() == '(0 resultados)':
                    print("Sin resultado")
                    break

            links = {link["href"] for link in sopa_review.findAll("a", class_="font-size-sm", href=True)}
        
        except Exception as e:
            print(f"Excepción al obtener la página {numero_pagina}: {e}")
        
        datos_link = [s_limpieza.procesar_link(link) for link in links]

        for dato in datos_link:
            s_limpieza.insertar_en_bd([dato]) 
            print(f"{dato} Insertado con éxito")

        
        
        sleep(2)


Una vez escrapeados los modelos vamos a ver las características principales de cada uno:

In [5]:
conn = s_sql.crear_conexion(bdd_local=True)
query = '''select marca,modelo,anio, link 
from public.modelos c ;'''

cur = conn.cursor()

cur.execute(query)
conn.commit()

model_ids = cur.fetchall()
conn.close()

Conexión creada con éxito


In [6]:
model_ids

[('Renault',
  'Scenic',
  2024,
  '/coches/renault/scenic/2024/estandar/informacion'),
 ('Peugeot', '5008', 2025, '/coches/peugeot/5008/2025/estandar/informacion'),
 ('Lexus', 'Es', 2026, '/coches/lexus/es/2026/estandar/informacion'),
 ('Livan', 'X6_pro', 2023, '/coches/livan/x6-pro/2023/estandar/informacion'),
 ('Denza', 'Z9gt', 2025, '/coches/denza/z9gt/2025/estandar/informacion'),
 ('Citroen', 'C3', 2024, '/coches/citroen/c3/2024/estandar/informacion'),
 ('Smart', '1', 2023, '/coches/smart/1/2023/estandar/informacion'),
 ('Ford',
  'Mustang_mach_e',
  2020,
  '/coches/ford/mustang/2020/suv/mach-e/informacion'),
 ('Volvo', 'Xc60', 2025, '/coches/volvo/xc60/2025/estandar/informacion'),
 ('Mazda', 'Cx_6e', 2026, '/coches/mazda/cx-6e/2026/estandar/informacion'),
 ('Kia', 'Ev3', 2025, '/coches/kia/ev3/2025/estandar/informacion'),
 ('Omoda', '9', 2025, '/coches/omoda/9/2025/estandar/informacion'),
 ('Omoda', '3', 2026, '/coches/omoda/3/2026/estandar/informacion'),
 ('Volkswagen',
  'Tayr

In [6]:
conn = s_sql.crear_conexion()
conn.rollback()
for datos_coche in model_ids:

    url_coche = f"https://www.km77.com{datos_coche[-1]}"
    print(url_coche)
    
    res_coche = requests.get(url_coche)
    print(res_coche.status_code)
    if res_coche.status_code == 200:
        sopa_carro = BeautifulSoup(res_coche.content,"html.parser")
        tablas = sopa_carro.find_all("table", class_="table table-sm table-hover mb-1")

        if tablas == []:
            pass
        else:
                #Esto es para sacar los numeros de la tabla que el formato es un poco extraño
            lista_caracteristicas = [row.get_text() for row in tablas[0].find_all("tr")]

            if s_limpieza.buscar_palabra(lista_caracteristicas,"Longitud"):
                    result_long = lista_caracteristicas[s_limpieza.buscar_palabra(lista_caracteristicas,"Longitud")]
                    longitud = re.findall(r"\d+,\d+", result_long)
            else:
                    longitud = None

            if s_limpieza.buscar_palabra(lista_caracteristicas,"maletero"):
                    result_maletero = lista_caracteristicas[s_limpieza.buscar_palabra(lista_caracteristicas,"maletero")]
                    maletero = re.findall(r"\d+",result_maletero)
            else:
                        maletero = None


            if s_limpieza.buscar_palabra(lista_caracteristicas,"Potencia"):
                            
                            potencia = lista_caracteristicas[s_limpieza.buscar_palabra(lista_caracteristicas,"Potencia")]
                            try:
                                pot_min =re.findall(r"\d+",potencia)[0]
                                pot_max =re.findall(r"\d+",potencia)[-1]
                            except IndexError:
                                    pass
                                                                    
            else:
                        pot_max = None
                        pot_min = None

            if s_limpieza.buscar_palabra(lista_caracteristicas,"Caja de cambios"):
                            cambios = lista_caracteristicas[s_limpieza.buscar_palabra(lista_caracteristicas,"Caja de cambios")]
                            caja_cambios = re.findall(r"Man|Aut", cambios)
            else:
                        caja_cambios = None

            if s_limpieza.buscar_palabra(lista_caracteristicas,"Tracción"):
                            tracc = lista_caracteristicas[s_limpieza.buscar_palabra(lista_caracteristicas,"Tracción")]
                            tracción = re.split(r"n", tracc)[1]
            else:
                        tracción = None
                        
            if s_limpieza.buscar_palabra(lista_caracteristicas,"Combustible"):
                            gasofa = lista_caracteristicas[s_limpieza.buscar_palabra(lista_caracteristicas,"Combustible")]
                            combustible = re.findall(r"[G|D]", gasofa)
            else:
                        combustible = None
            

            dic_info_carro = [{
                    
                    "Precio_base": [row.get_text() for row in tablas[0].find_all("tr")][0].split("\n")[1].split("€")[0],
                    "Precio_max": [row.get_text() for row in tablas[0].find_all("tr")][0].split("\n")[2].split("€")[0],
                    "Fecha_inicio": [row.get_text() for row in tablas[0].find_all("tr")][2].split("\n")[1].split("-")[0],
                    "Fecha_fin": [row.get_text() for row in tablas[0].find_all("tr")][2].split("\n")[1].split("-")[-1],
                    "Longitud": longitud,
                    "Volumen_maletero": maletero,
                    "Potencia_minima": pot_min,
                    "Potencia_maxima": pot_max,
                    "Caja_cambios": caja_cambios,
                    "Tipo_traccion": tracción,
                    "Combustible": [combustible if len(tablas[0].find_all("tr")) == 11 else "Hibrido/Eléctrico"]
                    }]
            
            df_info_carro = pd.DataFrame(dic_info_carro)

            df_info_carro = s_limpieza.normalizar_columnas_caracteristicas(df_info_carro)
            df_info_carro = s_limpieza.expandir_columnas_caracteristicas(df_info_carro)
            df_info_carro[["marca","modelo","anio"]] = (datos_coche[0],datos_coche[1],datos_coche[2])
            df_info_carro = s_limpieza.estandarizar_df_caracteristicas(df_info_carro)
        



            cur = conn.cursor()

            for row in df_info_carro.itertuples(index=False):
                    try:
                        cur.execute("""
                        INSERT INTO caracteristicas ("marca","modelo","anio","precio_base","precio_max","fecha_inicio","fecha_fin","potencia_minima","potencia_maxima","lon_0","lon_1","maletero_0","maletero_1","cambio_man","cambio_aut","tracc_del","tracc_total","tracc_tras","diesel","gasolina","electrificado")
                        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
                    
                        ON CONFLICT (marca, modelo, anio)
                        DO UPDATE SET

                        precio_base = EXCLUDED.precio_base,
                        precio_max = EXCLUDED.precio_max,
                        fecha_inicio = EXCLUDED.fecha_inicio,
                        fecha_fin = EXCLUDED.fecha_fin,
                        potencia_minima = EXCLUDED.potencia_minima,
                        potencia_maxima = EXCLUDED.potencia_maxima,
                        lon_0 = EXCLUDED.lon_0,
                        lon_1 = EXCLUDED.lon_1,
                        maletero_0 = EXCLUDED.maletero_0,
                        maletero_1 = EXCLUDED.maletero_1,
                        cambio_man = EXCLUDED.cambio_man,
                        cambio_aut = EXCLUDED.cambio_aut,
                        tracc_del = EXCLUDED.tracc_del,
                        tracc_total = EXCLUDED.tracc_total,
                        tracc_tras = EXCLUDED.tracc_tras,
                        diesel = EXCLUDED.diesel,
                        gasolina = EXCLUDED.gasolina,
                        electrificado = EXCLUDED.electrificado;                                    
                        """, 
                        (row.marca,
                            row.modelo,
                            row.anio,
                            row.Precio_base if type(row.Precio_base) == int else 0,
                            row.Precio_max if type(row.Precio_max) == int else 0,
                            row.Fecha_inicio if pd.notna(row.Fecha_inicio) else None,
                            row.Fecha_fin if pd.notna(row.Fecha_fin) else None,
                            row.Potencia_minima if pd.notna(row.Potencia_minima) else 0,
                            row.Potencia_maxima if pd.notna(row.Potencia_maxima) else 0,
                            row.Lon_0 if pd.notna(row.Lon_0) else 0,
                            row.Lon_1 if pd.notna(row.Lon_1) else 0,
                            row.Maletero_0 if pd.notna(row.Maletero_0) else 0,
                            row.Maletero_1 if pd.notna(row.Maletero_1) else 0,
                            row.Cambio_man if pd.notna(row.Cambio_man) else 0,
                            row.Cambio_aut if pd.notna(row.Cambio_aut) else 0,
                            row.Tracc_del if pd.notna(row.Tracc_del) else 0,
                            row.Tracc_total if pd.notna(row.Tracc_total) else 0,
                            row.Tracc_tras if pd.notna(row.Tracc_tras) else 0,
                            row.Diesel if pd.notna(row.Diesel) else 0,
                            row.Gasolina if pd.notna(row.Gasolina) else 0,
                            row.Electrificado if pd.notna(row.Electrificado) else 0)),
                        
                        conn.commit()
                        print(f"Fila {row.modelo} insertada correctamente")
                        sleep(2)

                    except NameError as e:
                        print(f"Error insertando {e}")
                        pass
             
            

      
    else:
        print(f"Este no arranca ({model_ids[-1]})")
        conn.close()
        break

Conexión creada con éxito
https://www.km77.com/coches/renault/scenic/2024/estandar/informacion
200
Fila Scenic insertada correctamente
https://www.km77.com/coches/peugeot/5008/2025/estandar/informacion
200
Fila 5008 insertada correctamente
https://www.km77.com/coches/lexus/es/2026/estandar/informacion
200
Fila Es insertada correctamente
https://www.km77.com/coches/livan/x6-pro/2023/estandar/informacion
200
Fila X6_pro insertada correctamente
https://www.km77.com/coches/denza/z9gt/2025/estandar/informacion
200
Fila Z9gt insertada correctamente
https://www.km77.com/coches/citroen/c3/2024/estandar/informacion
200
Fila C3 insertada correctamente
https://www.km77.com/coches/smart/1/2023/estandar/informacion
200
Fila 1 insertada correctamente
https://www.km77.com/coches/ford/mustang/2020/suv/mach-e/informacion
200
Fila Mustang_mach_e insertada correctamente
https://www.km77.com/coches/volvo/xc60/2025/estandar/informacion


KeyboardInterrupt: 

In [15]:
len(model_ids)

100

Por modelos


In [ ]:
df_categorias_coche = pd.DataFrame()
dic_referencia = {}

#Extraemos los links de los modelos de la gama coche:
for datos_coche in model_ids:
    url = f"https://www.km77.com{datos_coche[-1]}"


    url_separada = url.split("/")
    url_separada[-1] = "datos"
    url_reconstruida = "/".join(url_separada)
    print(f"Buscando versiones de: {url_reconstruida}")

    pagina_modelos = requests.get(url_reconstruida)
    sopa_modelos = BeautifulSoup(pagina_modelos.content,"html.parser")
    links_modelos = sopa_modelos.find_all('a', class_='vehicle-link')
    lista_links_modelos = [i["href"] for i in links_modelos]

    dic_referencia.update({datos_coche[-1]:[m.getText().strip() for m in links_modelos]})

    # Una vez tenemos los modelos, extraemos los datos en las tablas con la información necesaria y los procesamos
    lista_tabla = []

    for link in lista_links_modelos:
        link = "https://www.km77.com" + link
        print(f"link modelo y versión: {link}")
        res_link = requests.get(link)
        print(res_link.status_code)
        sopa_link = BeautifulSoup(res_link.content,"html.parser")
        tabla = sopa_link.find_all("table")
        # lista_tabla.append(tabla)

        # print(tabla)

        nombre_modelo = (link.split("/")[-2])
    #Nos aseguramos de que estén todos los contenidos:
        # if len(tabla) == 11:
        try:
            print (f"Analizando {nombre_modelo}")

            #Extraemos los datos importantes de las categorías del coche

            diccionario_cats = {}
            diccionario_cats.update({"Nombre" : nombre_modelo})

            for i,elemento in enumerate(tabla[:-1]):
                categoria =  ([row.get_text().strip() for row in tabla[i].find_all("tr")])

                # print (categoria)
                # print("*"*3)
                
            #Limpiamos los datos y creamos un diccionario
                for subcat in categoria:
                    #En caso de que estemos en la sentencia de precios normalizamos primero el contenido
                    if i == 0:
                        # print (f"subcat: {subcat}")
                        # print("-"*3)
                        subcat = re.findall(r'(.*?)(\d[\d\.,]*)', subcat)[0]

                        lista_limpia = [item for item in subcat]

                    else:    
                        partes = re.split(r"\n", subcat) 
                        lista_limpia = [item.strip() for item in partes if item.strip()]

                    # print(lista_limpia)
                    # print("*"*3)
                    diccionario_subcategorias = {}
                    if len(lista_limpia) == 2:
                        diccionario_subcategorias[lista_limpia[0]] = lista_limpia[1]

            #Limpiamos las unidades del diccionario y pasamos a df
                        diccionario_subcategorias = s_limpieza.limpiar_valores_diccionario(diccionario_subcategorias)

                lista_x = []
                        diccionario_cats.update(diccionario_subcategorias)   
                        df_modelo = pd.DataFrame([diccionario_cats])   
            df_categorias_coche = pd.concat([df_categorias_coche, df_modelo],axis = 0)
        
        except IndexError as e:
            print(e)
            pass

print(f"{df_categorias_coche.shape[0]} VERSIONES EXTRAÍDAS")

Buscando versiones de: https://www.km77.com/coches/renault/scenic/2024/estandar/datos
link modelo y versión: https://www.km77.com/coches/renault/scenic/2024/estandar/electric/scenic-e-tech-100-electrico-esprit-alpine-160-kw-220-cv-gran-autonomia/datos
200
Analizando scenic-e-tech-100-electrico-esprit-alpine-160-kw-220-cv-gran-autonomia
link modelo y versión: https://www.km77.com/coches/renault/scenic/2024/estandar/electric/scenic-e-tech-electric-autonomia-estandar2/datos
200
Analizando scenic-e-tech-electric-autonomia-estandar2
link modelo y versión: https://www.km77.com/coches/renault/scenic/2024/estandar/electric/scenic-e-tech-100-electrico-iconic-160-kw-220-cv-gran-autonomia/datos
200
Analizando scenic-e-tech-100-electrico-iconic-160-kw-220-cv-gran-autonomia
link modelo y versión: https://www.km77.com/coches/renault/scenic/2024/estandar/electric/scenic-e-tech-100-electrico-techno-125-kw-170-cv-autonomia-confort/datos
200
Analizando scenic-e-tech-100-electrico-techno-125-kw-170-cv-au

KeyboardInterrupt: 

QUEDA DIVIDIR LO DE ARRIBA EN DFS MAS PEQUEÑOS POR CATEGORIA


Tabulamos datos


In [17]:
diccionario_cats

{'Nombre': 'caravelle-batalla-larga-20-tdi-110-kw-150-cv-aut',
 'Precio (con descuento y equipamiento seleccionado)': '56640',
 'Descuento oficial': '5545',
 'Precio sin impuestos': '45808',
 'IVA': '21',
 'Impuesto de matriculación': '14.75',
 'Tarifa de': '03',
 'Velocidad máxima': '175',
 'Aceleración 0-100 km/h': '14.7',
 'Combinado': '8',
 'Velocidad baja': '10.7',
 'Velocidad media': '7.7',
 'Velocidad alta': '6.7',
 'Velocidad muy alta': '8.3',
 'Emisiones de CO₂ WLTP': '210',
 'Normativa de emisiones': '6',
 'Distintivo ambiental DGT': 'C',
 'Tipo de Carrocería': 'Vehículo comercial',
 'Número de puertas': '4',
 'Longitud': '5450',
 'Anchura': '2032',
 'Altura': '1978',
 'Batalla': '3500',
 'Peso': '2291',
 'Gasóleo': '55',
 'AdBlue': '20',
 'Filtro de partículas': 'Sí',
 'Volumen mínimo con tres filas de asientos disponibles': 'No disponible',
 'Número de plazas': '8',
 'Distribución de asientos': '2',
 'Potencia máxima': '150',
 'Par máximo': '360',
 'Propósito': 'Impulsar el

In [8]:
pd.set_option('display.max_columns', 500)
df_categorias_coche.sample()

,Nombre,Precio (con descuento y equipamiento seleccionado),Descuento oficial,Precio sin impuestos,IVA,Impuesto de matriculación,Tarifa de,Velocidad máxima,Aceleración 0-100 km/h,Combinado batería cargada,Autonomía eléctrica WLTP,Emisiones de CO₂ WLTP,Normativa de emisiones,Distintivo ambiental DGT,Tipo de Carrocería,Número de puertas,Longitud,Anchura,Altura,Batalla,Coeficiente Cx,Superficie frontal,Factor de resistencia,Peso,Volumen con una fila de asientos disponible,Volumen mínimo con dos filas de asientos disponibles,Número de plazas,Distribución de asientos,Altura libre,Potencia máxima,Par máximo,Propósito,Revoluciones potencia máxima,Revoluciones par máximo,Ubicación,Tensión nominal,Tipo,Capacidad,Capacidad útil,Potencia de recarga máxima en C.C.,Potencia de recarga máxima en C.A.,"Tiempo de recarga total a 7,4 kW",Tiempo de recarga total a 11 kW,Tiempo de recarga total a 22 kW,Tiempo de recarga del 10 al 80 % en C.C,Tracción,Caja de cambios,Número de velocidades,Tipo de mando,Tipo de Embrague,Tipo de mecanismo,Estructura suspensión delantera,Muelle suspensión delantera,Estructura suspensión trasera,Muelle suspensión trasera,Barra estabilizadora delantera,Barra estabilizadora trasera,Tipo de frenos delanteros,Diámetro de frenos delanteros,Tipo de frenos traseros,Diámetro de frenos traseros,Tipo de asistencia,Asistencia en función de la velocidad,Desmultiplicacion en función de la velocidad,Desmultiplicación no lineal,Desmultiplicación de la dirección,Dirección a las cuatro ruedas,Diámetro de giro entre bordillos,Vueltas de volante entre topes,Neumáticos delanteros,Neumáticos traseros,Llantas delanteras,Llantas Traseras,Vía delantera,Vía trasera,Combinado,Velocidad baja,Velocidad media,Velocidad alta,Velocidad muy alta,Gasolina,Volumen mínimo con tres filas de asientos disponibles,Combustible,Situación,Número de cilindros,Disposición de los cilindros,Material del bloque,Material de la culata,Diámetro,Carrera,Cilindrada,Válvulas por cilindro,Tipo de distribución,Alimentación,"Automatismo de parada y arranque del motor (""Stop/Start"")","Tiempo de recarga total a 3,7 kW",Ángulo de entrada,Ángulo de salida,Euro,Filtro de partículas,Aceleración,,,Relación de compresión,Volumen máximo con dos filas de asientos disponibles,Volumen del segundo maletero,Ángulo ventral,Diámetro de giro entre paredes,Altura de vadeo,1ª,2ª,3ª,4ª,5ª,6ª,7ª,8ª,Marcha atrás,"Tiempo de recarga total a 2,3 kW",Volumen máximo con tres filas de asientos disponibles,Gasóleo,AdBlue,Volumen del maletero
0,golf-r-line-15-tsi-110-kw-150-cv,35925,3125,31054,21,4.75,02,224,8.6,NaN,NaN,127,6,C,Turismo,5,4282,1789,1483,2620,No disponible,No disponible,NaN,1346,1237,381,5,2,NaN,150,250,Impulsar el vehículo,5000,1500,NaN,NaN,Cremallera,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Delantera,Manual,6,No disponible,Embrague monodisco en seco,Pares de engranajes,Tipo McPherson,Resorte helicoidal,Paralelogramo deformable,Resorte helicoidal,Sí,Sí,Disco ventilado,NaN,Disco,NaN,Eléctrica,Sí,No,No,NaN,No,NaN,NaN,225,225,75,75,1534,1502,5.6,7.2,5.6,4.9,5.6,50,NaN,Gasolina,Delantero transversal,4,En línea,Aluminio,Aluminio,74.5,85.9,1498,4,Dos árboles de levas en la culata,,Sí,NaN,NaN,NaN,NaN,Sí,NaN,NaN,NaN,10.5,NaN,NaN,NaN,11.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Extraccion de las descripciones

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")  # evita que se abrán las ventanas del driver
driver = webdriver.Chrome(options=chrome_options)

for datos_coche in model_ids:
    url = f"https://www.km77.com{datos_coche[-1]}"
    print(url)
    #entramos en la pagina
    driver.get(url)
    # Maximizamos a pantalla completa
    driver.maximize_window()

    # Encontrar y hacer clic en el botón de aceptar cookies
    try:
        driver.find_element("xpath", "/html/body/div[1]/div/div/div/div/div/div[3]/button[2]").click()
        print("Botón de aceptar cookies clicado con éxito.")
    except:
        print("No se pudo encontrar el botón de aceptar cookies o hacer clic en él.")
        pass

    #Aquí vamos a coger toda la información de la descripción
    # driver.execute_script("window. scrollTo(0, document. body. scrollHeight);")
    sleep(5)
    # #Cogemos todo el los párrafos que sean texto
    try:
        spans = driver.find_elements(By.CLASS_NAME, "texto")
        lista_textos = []
        for span in spans:
                lista_textos.append(span.text)
                print("textos extraídos con éxito")

    except:
        pass

    lista_textos = []

    rurl = requests.get(url)
    sopa_text = BeautifulSoup(rurl.content, 'html.parser')

    # textos_fortalezas = soup.find_all('strong')
    # lista_fortalezas.append([texto.get_text() for texto in textos_fortalezas])

    div_content = sopa_text.find_all('div', class_='mainbar col-12 col-lg min-width-0')
    lista_textos.append([texto.get_text() for texto in div_content])

    lista_textos = ",".join(lista_textos[0])

        

    ##Insertamos en SQL
    conn = s_sql.crear_conexion()
    cur = conn.cursor()
    list_textos = ",".join(lista_textos)
    dic_texto = {"marca": datos_coche[0],
                 "modelo": datos_coche[1],
                 "anio":datos_coche[2],
                 "reportaje":lista_textos}
    

    try:
            cur.execute("""
            INSERT INTO reviews(marca,modelo,anio,reportaje)
            VALUES(%s,%s,%s,%s)
            ON CONFLICT (marca,modelo,anio) DO UPDATE SET
            reportaje = EXCLUDED.reportaje""",(dic_texto.get("marca"),dic_texto.get("modelo"),dic_texto.get("anio"),dic_texto.get("reportaje")))

            conn.commit()
            print(f"Fila {'modelo'} insertada correctamente")

            sleep(10)
    except NameError as e:
            pass                
    
driver.quit()
conn.close()    


https://www.km77.com/coches/volkswagen/tayron/2025/estandar/informacion
No se pudo encontrar el botón de aceptar cookies o hacer clic en él.
Conexión creada con éxito
Fila modelo insertada correctamente
https://www.km77.com/coches/livan/x6-pro/2023/estandar/informacion
No se pudo encontrar el botón de aceptar cookies o hacer clic en él.
Conexión creada con éxito
Fila modelo insertada correctamente
https://www.km77.com/coches/byd/atto-3/2023/estandar/informacion
No se pudo encontrar el botón de aceptar cookies o hacer clic en él.
Conexión creada con éxito
Fila modelo insertada correctamente
https://www.km77.com/coches/lexus/ux/2023/estandar/informacion
No se pudo encontrar el botón de aceptar cookies o hacer clic en él.
Conexión creada con éxito
Fila modelo insertada correctamente
https://www.km77.com/coches/denza/z9gt/2025/estandar/informacion
No se pudo encontrar el botón de aceptar cookies o hacer clic en él.
Conexión creada con éxito
Fila modelo insertada correctamente
https://www.k